In [3]:
import tensorflow as tf
import keras
from keras import layers
import gymnasium as gym
import ale_py
from gymnasium.wrappers.atari_preprocessing import AtariPreprocessing
from gymnasium.wrappers.frame_stack import FrameStack
import numpy as np

# Register Atari environments to ensure compatibility
# gym.register_envs(ale_py)

# # Create the Space Invaders environment with frame preprocessing
# env = gym.make("SpaceInvadersNoFrameskip-v4", render_mode= "rgb_array")
# env = AtariPreprocessing(env)  # Preprocess frames (grayscale, resize)
# env = FrameStack(env, 4)  # Stack 4 consecutive frames for temporal information
# trigger = lambda t:t % 1000 == 0
# env = gym.wrappers.RecordVideo(env, video_folder="videos", episode_trigger=trigger, disable_logger=True)
# # Determine the number of possible actions in Space Invaders
# num_actions = env.action_space.n

# def create_q_model():
#     """
#     Create a Convolutional Neural Network for Q-Learning
#     Architecture inspired by DeepMind's original DQN paper
#     """
#     return keras.Sequential([
#         # Input layer expecting 4 stacked grayscale frames of 84x84 pixels
#         keras.Input(shape=(84, 84, 4)),
        
#         # First convolutional layer to extract low-level features
#         layers.Conv2D(32, kernel_size=8, strides=4, activation="relu"),
        
#         # Second convolutional layer for more complex feature extraction
#         layers.Conv2D(64, kernel_size=4, strides=2, activation="relu"),
        
#         # Third convolutional layer for fine-grained feature detection
#         layers.Conv2D(64, kernel_size=3, strides=1, activation="relu"),
        
#         # Flatten the 2D feature maps into a 1D vector
#         layers.Flatten(),
        
#         # Fully connected layer with 512 neurons for high-level reasoning
#         layers.Dense(512, activation="relu"),
        
#         # Output layer with linear activation to predict Q-values for each action
#         layers.Dense(num_actions, activation="linear")
#     ])

# # Create two neural networks: 
# # 1. Action network (model) for current Q-value predictions
# # 2. Target network (model_target) for stable training
# model = keras.models.load_model("spaceinvaders_qmodel_508.keras")
# model_target = keras.models.load_model("spaceinvaders_qmodel_508.keras")

# # Adam optimizer with learning rate and gradient clipping
# optimizer = keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0)

# # Initialize replay buffer and tracking variables
# action_history = []
# state_history = []
# state_next_history = []
# rewards_history = []
# episode_reward_history = []
# done_history = []

# # Tracking variables for training progress
# running_reward = 0
# episode_count = 508
# frame_count = 260000

# # Hyperparameters for Deep Q-Learning
# gamma = 0.99  # Discount factor for future rewards
# epsilon = 1.0  # Exploration rate
# epsilon_min = 0.1  # Minimum exploration probability
# epsilon_max = 1.0  # Maximum exploration probability
# epsilon_interval = epsilon_max - epsilon_min
# batch_size = 32  # Number of experiences to sample for training

# # Training control parameters
# max_episodes = 0  # 0 means train until a stopping condition is met
# max_frames = 1e7  # Maximum total frames to train
# epsilon_random_frames = 5  # Initial frames with random actions
# epsilon_greedy_frames = 1e6  # Frames to linearly decay exploration
# max_memory_length = 1e5  # Maximum size of replay buffer
# max_steps_per_episode = 10000  # Prevent infinite episodes
# update_after_actions = 4  # Update frequency for action network
# update_target_network = 10000  # Frequency of updating target network

# # Use Huber loss for more stable training
# loss_function = keras.losses.Huber()

# # Main training loop
# while True:
#     # Reset environment and get initial observation
#     observation, _ = env.reset()
#     state = np.array(observation)
#     episode_reward = 0

#     for timestep in range(1, max_steps_per_episode):
#         frame_count += 1

#         # Epsilon-greedy action selection
#         if frame_count < epsilon_random_frames or epsilon > np.random.rand(1)[0]:
#             # Take random action during initial exploration
#             action = env.action_space.sample()
#         else:
#             # Preprocess state for model prediction
#             state_tensor = keras.ops.convert_to_tensor(state)
#             # Transpose to match model's expected input shape
#             state_tensor = keras.ops.transpose(state_tensor, [1, 2, 0])
#             # Add batch dimension
#             state_tensor = keras.ops.expand_dims(state_tensor, 0)
            
#             # Predict best action
#             action_probs = model(state_tensor, training=False)
#             action = keras.ops.argmax(action_probs[0]).numpy()

#         # Decay exploration rate
#         epsilon -= epsilon_interval / epsilon_greedy_frames
#         epsilon = max(epsilon, epsilon_min)

#         # Take action in environment
#         state_next, reward, done, _, _ = env.step(action)
#         state_next = np.array(state_next)

#         episode_reward += reward

#         # Store experience in replay buffer
#         action_history.append(action)
#         state_history.append(state)
#         state_next_history.append(state_next)
#         done_history.append(done)
#         rewards_history.append(reward)
#         state = state_next

#         # Training step
#         if frame_count % update_after_actions == 0 and len(done_history) > batch_size:
#             # Sample random batch from replay buffer
#             indices = np.random.choice(range(len(done_history)), size=batch_size)

#             # Prepare batch data
#             state_sample = np.array([state_history[i] for i in indices])
#             state_next_sample = np.array([state_next_history[i] for i in indices])
#             rewards_sample = [rewards_history[i] for i in indices]
#             action_sample = [action_history[i] for i in indices]
            
#             # Convert done flags to tensor
#             done_sample = tf.convert_to_tensor([float(done_history[i]) for i in indices])

#             # Predict future rewards using target network
#             # Ensure correct input shape: (batch_size, height, width, frames)
#             state_next_sample_reshaped = np.transpose(state_next_sample, (0, 2, 3, 1))
#             future_rewards = model_target.predict(state_next_sample_reshaped, verbose=0)

#             # Calculate updated Q-values using Bellman equation
#             updated_q_values = rewards_sample + gamma * tf.reduce_max(future_rewards, axis=1)
#             # Adjust Q-values for terminal states
#             updated_q_values = updated_q_values * (1 - done_sample) - done_sample

#             # Create one-hot encoded action mask
#             masks = tf.one_hot(action_sample, num_actions)

#             # Compute gradients
#             with tf.GradientTape() as tape:
#                 # Predict Q-values for sampled states
#                 state_sample_reshaped = np.transpose(state_sample, (0, 2, 3, 1))
#                 q_values = model(state_sample_reshaped)
                
#                 # Select Q-values for taken actions
#                 q_action = tf.reduce_sum(tf.multiply(q_values, masks), axis=1)
                
#                 # Compute loss
#                 loss = loss_function(updated_q_values, q_action)

#             # Compute and apply gradients
#             grads = tape.gradient(loss, model.trainable_variables)
#             optimizer.apply_gradients(zip(grads, model.trainable_variables))

#         # Periodically update target network
#         if frame_count % update_target_network == 0:
#             model_target.set_weights(model.get_weights())
#             print(f"Best score of last 100: {np.max(episode_reward_history) if episode_reward_history else 0}, "
#                   f"running_reward: {running_reward} at episode {episode_count}, frame {frame_count}")
#             model.save(f"models/spaceinvaders_qmodel_{episode_count}.keras")

#         # Manage replay buffer size
#         if len(rewards_history) > max_memory_length:
#             del rewards_history[:1]
#             del state_history[:1]
#             del state_next_history[:1]
#             del action_history[:1]
#             del done_history[:1]

#         # End episode if done
#         if done:
#             break

#     # Track episode rewards
#     episode_reward_history.append(episode_reward)
#     if len(episode_reward_history) > 1000:
#         del episode_reward_history[:1]
#     running_reward = np.mean(episode_reward_history)

#     episode_count += 1
#     print(episode_count-1, episode_reward)

#     # Stopping conditions
#     if running_reward > 800:  # Adjust threshold for Space Invaders
#         print(f"Solved at episode {episode_count}!")
#         model.save(f"models/spaceinvaders_qmodel_solved.keras")
#         break

#     if (max_episodes > 0 and episode_count >= max_episodes):
#         print(f"Stopped at episode {episode_count}!")
#         break

#     if (max_frames > 0 and frame_count >= max_frames):
#         print(f"Stopped at frame {frame_count}!")
#         break

ModuleNotFoundError: No module named 'gymnasium'